## Monitoring wetland changes and their impact of malaria transmisson in the Zambezi Delta, Mozambique

In [ ]:
%matplotlib inline

import datacube
import seaborn as sns
import matplotlib.pyplot as plt
from datacube.utils import masking
from datacube.utils import geometry
from datacube.utils.geometry import CRS

from deafrica_tools.plotting import display_map, plot_wofs
from deafrica_tools.datahandling import wofs_fuser, mostcommon_crs

In [ ]:
# connect to the Digital Earth Africa data cube
dc = datacube.Datacube(app="Intro_WOfS")

In [ ]:
# check and list all the WOfS products available in Digital Earth Africa
products = dc.list_products()
display_columns = ['name', 'description']
dc_products = products[display_columns]
dc_products[dc_products['name'].str.contains("wofs_ls")]

In [ ]:
# Application parameters: location and time range 
# Location = Mopeia region within the Zambezi Delta in Mozambique
# Time range between 2000 and 2022 

lat, lon =  -17.9782, 35.7136
buffer = 0.245 # 30km buffer
time_range = ('2000', '2022')

#add lat,lon,buffer together to get bounding box
x = (lon-buffer, lon+buffer)
y =  (lat+buffer, lat-buffer)

In [ ]:
# View the location 
display_map(x=x, y=y)

In [ ]:
# Load WOfS layer
# Create a reusable query
query = {
    'x': x,
    'y': y,
    'time': ("2010"),
    'resolution': (-30, 30)
}

#get crs of location
output_crs = mostcommon_crs(dc=dc, product='wofs_ls', query=query)

# Load WOfS feature layers 
wofls= dc.load(product = 'wofs_ls',
               group_by="solar_day",
               fuse_func=wofs_fuser,
               output_crs = output_crs,
               collection_category="T1",
               **query)
print(wofls)

In [ ]:
# Classify different types of data in the area of interest
plot_wofs(wofls.water, col='time', size=4, col_wrap=4);

In [ ]:
# Keeping only dry, non-cloudy pixels
wofls_wet = masking.make_mask(wofls, wet=True)

# Plot output mask
wofls_wet.water.plot(col='time', size=4, col_wrap=4);

In [ ]:
# load WOfS all time summary 
wofs_alltime = dc.load(product='wofs_ls_summary_alltime',
               like=wofls.geobox)

print(wofs_alltime)

# plot the all time summary map
fig, ax = plt.subplots()
im = wofs_alltime.frequency.plot(ax=ax, cmap="mako_r", add_colorbar=False)
# Add colorbar with label
cbar = plt.colorbar(im, ax=ax, orientation='vertical', fraction=0.03, pad=0.02)
cbar.set_label('Frequency')
# Add title and axis labels
ax.set_title('WOfS All-Time Water Frequency (1985 - Present)', fontsize=12, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
# loading WOfS annual summary
wofs_annual = dc.load(product='wofs_ls_summary_annual',
               like=wofls.geobox,
               time='2001')

print(wofs_annual)

In [ ]:
wofs_annual.frequency.plot(size=4, cmap=sns.color_palette("mako_r", as_cmap=True))
plt.title('WOfS Annual Summary for 2000');

In [ ]:
# plot all the maps for the time range
def plot_wofs_annual_all(dc, wofls):
    years = range(2000, 2022)
    fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(18, 12))  
    axes = axes.flatten()

    for i, year in enumerate(years):
        wofs_annual = dc.load(product='wofs_ls_summary_annual', like=wofls.geobox, time=str(year))
        im = wofs_annual.frequency.plot(ax=axes[i], cmap=sns.color_palette("mako_r", as_cmap=True))
        axes[i].set_title(f'WOfS Annual Summary {year}', fontsize=10)
        axes[i].set_ylabel('Latitude')
        axes[i].set_xlabel('Longitude')
        
    # Remove extra subplots
    for ax in axes[len(years):]: ax.remove()
        
    plt.tight_layout()
    plt.show()

plot_wofs_annual_all(dc, wofls)